In [148]:
#%pip install BeautifulSoup4
#%pip install requests
#%pip install gensim==3.6.0


In [149]:
# Reads headlines.txt file into list 
def read_file(filename):
    with open(filename, 'r') as f:
        each_line = f.readlines()
        headlines = []
        for word in each_line:
            if word.isalpha():
                each_line = each_line.replace(word, word.lower())       
        for line in each_line:
            line = line.strip()
            if line:
                headlines.append(line)
        return headlines
    
# Reads populat_topics.txt file and returns list
def read_file_word(filename):
    with open(filename, 'r') as f:
        each_line = f.read().split()
        return each_line

# Read links file into list    
def read_links(filename):
    with open(filename, 'r') as l:
        links = [line.strip() for line in l]
        return links

In [150]:
# Gets the headlines that have the top words
words = read_file_word('popular_topics.txt')
headlines = read_file('headlines.txt')
popular_headlines = []
for word in words:
    for headline in headlines:
        if word in headline.split():
            popular_headlines.append(headline)
        else:
            continue

In [151]:
popular_headlines

['What is the moon phase today? Lunar phases 2023',
 "This is our 1st detailed look at Mars' most mysterious moon Deimos (photos)",
 'Tokyo company aims to be 1st business to put lander on moon',
 'A runaway black hole has been spotted fleeing a distant galaxy',
 'Star-forming activity in the interacting galaxy system NGC 5291 investigated with AstroSat',
 'Hubble spotlights barred spiral galaxy UGC 678',
 'Webb reveals early-universe prequel to huge galaxy cluster',
 'The first black hole portrait got sharper thanks to machine learning',
 'A runaway black hole has been spotted fleeing a distant galaxy',
 'Black hole takes multiple bites at the cherry',
 'Supermassive black hole movies in action',
 "Uncovering a star's demise: Supermassive black hole tears apart a giant star",
 "New baby exoplanet discovered thanks to the 'wake' it leaves around its parent star",
 'A neutron star collision may have emitted a fast radio burst',
 "Uncovering a star's demise: Supermassive black hole tears

In [152]:
# Find link for given headline and add it to list_links with headline
links = read_links('links.txt')
list_links = []
for x in popular_headlines:
    if x in headlines:
        index = headlines.index(x)
        link = links[index]
        list_links.append(link)
    else:
        print("Headline not found.")

In [153]:
list_links

['https://www.space.com/18880-moon-phases.html',
 'https://www.space.com/mars-moon-deimos-image-origin-story',
 'https://phys.org/news/2023-04-tokyo-company-aims-1st-business.html',
 'https://www.sciencenews.org/article/runaway-black-hole-galaxy-hubble',
 'https://phys.org/news/2023-04-star-forming-interacting-galaxy-ngc-astrosat.html',
 'https://phys.org/news/2023-04-hubble-spotlights-barred-spiral-galaxy.html',
 'https://phys.org/news/2023-04-webb-reveals-early-universe-prequel-huge.html',
 'https://www.sciencenews.org/article/first-black-hole-m87-machine-learning',
 'https://www.sciencenews.org/article/runaway-black-hole-galaxy-hubble',
 'https://www.nature.com/articles/s41550-023-01908-y',
 'https://www.nature.com/articles/s41550-023-01909-x',
 'https://phys.org/news/2023-04-uncovering-star-demise-supermassive-black.html',
 'https://www.space.com/baby-exoplanet-discovery-eso-very-large-telescope',
 'https://www.sciencenews.org/article/neutron-star-collision-fast-radio-burst',
 'htt

In [164]:
#Summarizes content from links (1m 10s)
import requests
from bs4 import BeautifulSoup
from gensim.summarization import summarize

summary_list = []
# Loop through each link
for link in list_links:
    # Download HTML content of link
    headers ={'User-Agent': 'Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract article content
    paragraphs = soup.find_all('p')
    article_text = '\n'.join([p.text for p in paragraphs])

    # Summarize article content
     # Summarize article content
    if (len(summarize(article_text, ratio=0.4, split=True)))==0:
        summary = summarize(article_text, ratio=0.4, split=True)
        # Append summary
        summary_list.append(summary[0])
    else:
        summary = summarize(article_text, ratio=0.4, split=True)
        # Append summary
        summary_list.append(summary[0])

In [165]:
# Returns nouns from string
def return_noun(line):
    # function to test if something is a noun
    is_noun = lambda pos: pos[:2] == 'NN'
    # do the nlp stuff
    tokenized = nltk.word_tokenize(line)
    nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
    return nouns

In [166]:
#%pip install nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/rhessa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rhessa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [167]:
#%pip install xlrd
import pandas as pd
import difflib

# Read the first Excel file into a dataframe
df1 = pd.read_excel('NGC.xlsx', sheet_name='NGC')

# Read the second Excel file into a dataframe
df2 = pd.read_excel('mesr-mas.xls', sheet_name='Messier Objects', skiprows = 8, nrows=110, usecols=range(2))


In [170]:
# Create lists for each Messier and NGC objects
messier1 = df2.iloc[:,0].tolist()
NGC2 = df2.iloc[:,1].dropna().tolist()
NGC1 = df1.iloc[:,24].dropna().tolist()

# Combine two NGC to find missing elements
NGC_str = list(set(NGC1).union(set(NGC2)))

# Convert to string plus catalog identifier
messier = ['M' + str(element) for element in messier1]
NGC = ['NGC ' + str(element).rstrip('.0') for element in NGC_str]

# Create one total list
total_list = messier+NGC


In [171]:
# Compare each summary to the list of Messier objects
import re
object_ls = []
for object_name in total_list:
    for summary in summary_list:
        if re.search(r'\b' + object_name + r'\b', summary):
            print(object_name, "found in summary:", summary)
            object_ls.append(object_name)

M87 found in summary: Scientists used machine learning to create a sharper image of the supermassive black hole in the galaxy M87 (right), compared with the original image (left).


In [172]:
object_ls

['M87']